In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 64#32
block_size = 256#8
max_iters = 5000
eval_interval = 500#300
learning_rate = 3e-4#1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
eval_iters = 200
n_emb = 384#32
n_head = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)

# read and inspect
with open('input.txt', 'r', encoding= 'utf-8') as f:
    text = f.read()


# vocab = all unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a Mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i : ch for i, ch in enumerate(chars)}

#Most Basic and Easy tokenizer
encode = lambda s: [stoi[ch] for ch in s]   # take a string and outputs a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # take list of integers and outputs a string
data = torch.tensor(encode(text), dtype = torch.long)
# Let's Split the Dataset into Training and validation
n = int(0.9 * len(data))  # first 90% will be train , rest is validation
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:i+ block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()

    model.train()
    return out

class Head(nn.Module):
    """One head of self Attention """
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_emb,head_size, bias= False)
        self.query = nn.Linear(n_emb,head_size, bias= False)
        self.value = nn.Linear(n_emb,head_size, bias= False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        # by convention tril is not a parameter , it is a buffer
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) #(B,T,C)
        # compute attention scores("affinities")
        wei = q @ k.transpose(-2,-1) * C** -0.5 # (B,T,C) @ ( B,C,T) ---> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] ==0 , float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        # perform the weighted aggregration of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """Multiple Head of self attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_emb,n_emb) # projection = simple a Linear Transformation of output
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
        # we are simply concatenating all of the outputs (-1 means channels , we are simply conatenating channels)

class FeedForward(nn.Module):
    """a simple Linear Layer followed by a Non-Linearity"""
    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb,4*n_emb),
            nn.ReLU(),
            nn.Linear(4* n_emb,n_emb), # projection layer just like we wrote it in MultiHeadAttention
            nn.Dropout(dropout),
        )

    def forward(self,x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer Block : communication(self attention) followed by Computation(Feed forward)"""

    def __init__(self, n_emb, n_head):
        # n_emb = Number of Embeddings , n_head = Number of Heads in Self-attention
        super().__init__()
        head_size = n_emb // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_emb)
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)


    def forward(self, x):
        # x = x + self.sa(x) # x + means skip connections
        # x = x + self.ffwd(x)
        # we ar applying LayerNorm before the communication
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_emb)
        self.position_embedding_table = nn.Embedding(block_size,n_emb)

        # self.sa_head = Head(n_emb)  # self-attention head
        # originally we have n_emb = 32 , now 4 heads and 32 will be divided for ease of calculation in parallel
        # hence in output we are concatenating all outputs and it will make up to 32
        # self.sa_head = MultiHeadAttention(4,n_emb//4)  # now 4 heads of 8 dimensional self-attention
        # self.ffwd = FeedForward(n_emb)
        # self.block = nn.Sequential(
        #     Block(n_emb,n_head = 4),
        #     Block(n_emb,n_head = 4),
        #     Block(n_emb,n_head= 4),
        #     nn.LayerNorm(n_emb)
        # )
        # we can also write block more Nicely
        self.block = nn.Sequential(*[Block(n_emb, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_emb) # final layer norm
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self,idx , targets= None):
        B,T = idx.shape
        #idx(input index) and targets(output index) are both (B,T) tensors of integers
        token_emb = self.token_embedding_table(idx) # (B,T,C) (batch, time , channel)
        pos_emb = self.position_embedding_table(torch.arange(T,device= device)) # T,C
        x = token_emb + pos_emb
        x = self.block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)


        if targets == None:
            loss= None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits,targets) # -> this will not work because logits are (B,T,C) and Cross entropy wants (B,C)
        # (B,T,C) tells us each batch has T times value(input value = block size) and each T has C embedding values
        return logits,loss

    def generate(self,idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to last block_size tokens
            idx_cond = idx[:,-block_size:] # so that it won't get out of scope
            # get the predictions
            logits, loss = self(idx_cond)
            # in prediction of bi-gram model , we only care about one element to predict next one
            logits = logits[:,-1,:]  # becomes (B,C) , since (B,T,C) now T only have one element
            # apply softmax to get probabilities
            probs = F.softmax(logits , dim = 1)
            # sample from the distribution
            idx_next = torch.multinomial(probs,num_samples = 1) # (B,T)
            # append sampled index to the running sequence
            idx = torch.cat((idx,idx_next), dim = 1) # (B,T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

# Let's Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

for iter in range(max_iters):

    #every once in a while evaluate the loss on train and val sets and print
    if iter % eval_interval == 0:
        losses = estimate_loss() # loss can be noisy because each Batch is doing calculation in parallel
        # that's why we are calculating average of both the losses in estimate_loss() function
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss{losses['val']:.4f}  ")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits , loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)  # Most Imp step to remember otherwise grad will sum up
    loss.backward()
    optimizer.step()  # update the parameters

# generate from the Model
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context,max_new_tokens=200)[0].tolist()))

step 0: train loss 4.2849, val loss4.2823  
step 500: train loss 1.6774, val loss1.8435  
step 1000: train loss 1.3518, val loss1.5843  
step 1500: train loss 1.2349, val loss1.5048  
step 2000: train loss 1.1519, val loss1.4862  
step 2500: train loss 1.0872, val loss1.4836  
step 3000: train loss 1.0267, val loss1.5018  
step 3500: train loss 0.9672, val loss1.5243  
step 4000: train loss 0.9077, val loss1.5528  
step 4500: train loss 0.8489, val loss1.5667  

But with a hungry and unstain winged virtues,
His white shame wounded; for his unalways,
After many one in the hour, no more heart.
I mean tie he speak. Hall you, tassians
The hatre that you most most
